<a href="https://colab.research.google.com/github/pri2k/BTP/blob/main/Exploringlib/classification2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uploading files

In [24]:
# Step 1: Upload the file
from google.colab import files
uploaded = files.upload()

# Get the filename
filename = list(uploaded.keys())[0]

# Read and join all paragraphs as one big text
with open(filename, 'r', encoding='utf-8') as f:
    content = f.read()

# Step 2: Split lines and filter out '---------' and blank lines
lines = content.strip().splitlines()
filtered_lines = [line.strip() for line in lines if not line.strip().startswith('-') and line.strip() != '']

# Step 3: Join into paragraphs and flatten
combined_text = ' '.join(filtered_lines)


# lines = content.strip().splitlines()
# filtered_lines = [line.strip() for line in lines if not line.strip().startswith('-') and line.strip() != '']
# combined_text = ' '.join(filtered_lines)


# # Split on paragraph breaks (double newline), then join with space

# paragraphs = content.strip().split('\n\n')
# combined_text = ' '.join(paragraph.strip().replace('\n', ' ') for paragraph in paragraphs)

# print(f" Combined {len(paragraphs)} paragraphs into one block of {len(combined_text.split())} words.")


Saving harvey_weinstein to harvey_weinstein (1)
Saving demonetization_9 to demonetization_9 (1)
Saving catalan_independence to catalan_independence (1)
Saving ball_tampering to ball_tampering (1)


Installing requirements (you only need to run it one time when running the code for the first time)

In [25]:
!pip install rake-nltk yake
# !pip uninstall -y transformers
!pip install transformers --upgrade

!pip install --upgrade huggingface_hub
!pip install --upgrade keybert sentence-transformers

import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

import and extractors defining

In [26]:
from transformers import AutoTokenizer, AutoModel
!git lfs install
!git clone https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

Git LFS initialized.
fatal: destination path 'all-MiniLM-L6-v2' already exists and is not an empty directory.


In [27]:
# RAKE
from rake_nltk import Rake

# YAKE
import yake

# KeyBERT
from keybert import KeyBERT

# Initialize models
rake = Rake()
kw_extractor_yake = yake.KeywordExtractor(lan="en", top=10)
kw_model = KeyBERT(model="sentence-transformers/all-MiniLM-L6-v2")


functions for each extractor

In [28]:
# def extract_keywords_rake(text, top_n=100):
#     rake.extract_keywords_from_text(text)
#     return rake.get_ranked_phrases_with_scores()[:top_n]

def extract_keywords_rake(text, top_n=100, min_words=1, max_words=1):
    rake.extract_keywords_from_text(text)
    ranked_phrases = rake.get_ranked_phrases_with_scores()

    seen = set()
    filtered = []

    for score, phrase in ranked_phrases:
        phrase_clean = phrase.strip().lower()
        word_count = len(phrase_clean.split())
        if phrase_clean not in seen and min_words <= word_count <= max_words:
            filtered.append((score, phrase))
            seen.add(phrase_clean)
        if len(filtered) >= top_n:
            break

    return filtered


def extract_keywords_yake(text):
    return kw_extractor_yake.extract_keywords(text)

def extract_keywords_keybert(text):
    return kw_model.extract_keywords(
        text,
        top_n=100,
        stop_words='english',
        keyphrase_ngram_range=(1, 2)
    )


Calling the functions and printing

In [29]:
print("\n==============================")
print(" Extracting keywords from entire article collection\n")

print("---  RAKE Keywords ---")
for score, kw in extract_keywords_rake(combined_text):
    print(f"{kw} ({score:.2f})")

print("\n---  YAKE Keywords ---")
for kw, score in extract_keywords_yake(combined_text):
    print(f"{kw} ({score:.6f})")

print("\n---  KeyBERT Keywords ---")
for kw, score in extract_keywords_keybert(combined_text):
    print(f"{kw} ({score:.4f})")



 Extracting keywords from entire article collection

---  RAKE Keywords ---
empire (6.50)
claims (6.20)
saturday (5.67)
stoller (5.00)
1990 (5.00)
october (4.89)
expand (4.67)
salma (4.50)
chased (4.50)
17 (4.50)
16 (4.50)
wednesday (4.33)
since (4.33)
details (4.33)
weinsteins (4.25)
action (4.20)
bill (4.17)
friday (4.14)
director (4.12)
thursday (4.07)
tells (4.00)
takes (4.00)
oppress (4.00)
named (4.00)
editor (4.00)
chief (4.00)
actor (4.00)
24 (4.00)
23 (4.00)
tuesday (3.85)
monday (3.77)
figures (3.75)
variety (3.67)
investigators (3.67)
including (3.67)
published (3.62)
like (3.57)
mogul (3.56)
walk (3.50)
started (3.50)
spokeswoman (3.50)
safe (3.50)
president (3.50)
month (3.50)
mcgowan (3.50)
investigation (3.50)
dollars (3.50)
december (3.50)
confirmed (3.50)
complicated (3.50)
building (3.50)
prosecutors (3.45)
harvey (3.44)
evans (3.43)
times (3.40)
used (3.40)
magazine (3.38)
two (3.36)
‘ (3.36)
star (3.33)
mira (3.33)
manhattan (3.33)
makes (3.33)
kantor (3.33)
accoun

In [30]:
rake_keywords = extract_keywords_rake(combined_text)  # (score, keyword)
yake_keywords = extract_keywords_yake(combined_text)  # (keyword, score)
keybert_keywords = extract_keywords_keybert(combined_text)  # (keyword, score)
print("Succesfully stored")


Succesfully stored


Downloading the extracted keywords


In [31]:
import pandas as pd

# Convert to DataFrames
rake_df = pd.DataFrame(rake_keywords, columns=["Score", "Keyword"])
yake_df = pd.DataFrame(yake_keywords, columns=["Keyword", "Score"])
keybert_df = pd.DataFrame(keybert_keywords, columns=["Keyword", "Score"])

# Save to CSV files
rake_df.to_csv("Dem_rake_keywords.csv", index=False)
yake_df.to_csv("Dem_yake_keywords.csv", index=False)
keybert_df.to_csv("Dem_keybert_keywords.csv", index=False)

print("Keywords saved to CSV files.")

from google.colab import files

files.download("Dem_rake_keywords.csv")
files.download("Dem_yake_keywords.csv")
files.download("Dem_keybert_keywords.csv")



Keywords saved to CSV files.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CLASSIFY WHETHER TARGET OR NOT

In [32]:
def classify_keywords(keyword_list, method_keywords, threshold, method_name):
    if method_name == "RAKE":
        # RAKE gives (score, phrase), so we reverse it
        kw_dict = {kw.lower(): score for score, kw in method_keywords}
    else:
        kw_dict = {kw.lower(): score for kw, score in method_keywords}

    results = {}

    for keyword in keyword_list:
        keyword_lower = keyword.lower()
        found = False

        for phrase, score in kw_dict.items():
            if keyword_lower in phrase:
                # print(f"[DEBUG] {keyword_lower} matched phrase '{phrase}' with score {score}")
                if method_name in ["RAKE", "KeyBERT"]:
                    if score >= threshold:
                        found = True
                        break
                else:  # YAKE: lower is better
                    if round(score, 6) < round(threshold, 6):
                        found = True
                        break

        results[keyword] = found

    return results


For every new article make sure to change the keyword to test and the threshold values.

In [33]:
keywords_to_test = ["Catalan independence"]
raketest=["susana vera “ falling apart ” puigdemont appeared","mobile world congress","former parliament speaker carme forcadell"]
yaketest=["Spain","Catalan independence"]
keyberttest=["catalonia want independence","regional election catalonia","catalonians voted","spain called elections"]

rake_threshold = 30
yake_threshold = 0.000277  # e.g. raw score, lower means more important
keybert_threshold = 0.63

rakewords= classify_keywords(raketest, rake_keywords, rake_threshold, "RAKE")
yakewords= classify_keywords(yaketest, yake_keywords, yake_threshold, "YAKE")
keybertwords= classify_keywords(keyberttest, keybert_keywords, keybert_threshold, "KeyBERT")

print("Rake classification")
for word, is_target in rakewords.items():
    print(f"{word} → {is_target}")
print("\n")

print("Yake classification")
for word, is_target in yakewords.items():
    print(f"{word} → {is_target}")
print("\n")

print("Keybert classification")
for word, is_target in keybertwords.items():
    print(f"{word} → {is_target}")


Rake classification
susana vera “ falling apart ” puigdemont appeared → False
mobile world congress → False
former parliament speaker carme forcadell → False


Yake classification
Spain → False
Catalan independence → False


Keybert classification
catalonia want independence → False
regional election catalonia → False
catalonians voted → False
spain called elections → False
